In [ ]:
#历史转化率：根据广告分组，统计用户特征
item_count = groupby item count instance in train_data
item_dist_count_user = groupby item distinct count user in train_data 
item_y1_user_count = groupby item count user in train_data when y==1
item_y1_user_dist_count = groupby item distinct count user in train_data when y==1
item_y1_user_count_div_user_count = item_y1_user_count / item_count
item_y1_user_count_div_user_dist_count = item_y1_user_count / item_dist_count_user
item_y1_user_dist_count_div_user_dist_count = item_y1_user_dist_count / item_dist_count_user
    
#example
tf0 = item_group[["instance_id"]].count()
tf0.rename(columns={"instance_id":"item_count"},inplace=True)

tf1_1 = tf0.copy()
tf1_1["item_user_dist_count"] = item_group["user_id"].agg(lambda x:x.unique().size)
tf1_2 = item_y1_group[["user_id"]].count()
tf1_2.rename(columns={"user_id": "item_y1_user_count"},inplace=True)
tf1_2["item_y1_user_dist_count"] = item_y1_group["user_id"].agg(lambda x:x.unique().size)
tf1_1.reset_index(inplace=True)
tf1_2.reset_index(inplace=True)
tf1_tmp = pd.merge(left=tf1_1, right=tf1_2, how="left", on="item_id")
tf1_tmp["item_y1_user_count_div_user_count"] = tf1_tmp["item_y1_user_count"] / tf1_tmp["item_count"]
tf1_tmp["item_y1_user_count_div_user_dist_count"] = tf1_tmp["item_y1_user_count"] / tf1_tmp["item_user_dist_count"]
tf1_tmp["item_y1_user_dist_count_div_user_dist_count"] = tf1_tmp["item_y1_user_dist_count"] / tf1_tmp["item_user_dist_count"]
tf1_tmp.fillna(value=0, inplace=True)
tf1 = tf1_tmp.drop(labels="item_count", axis=1)

In [ ]:
#样本计数特征:根据用户分组，计数统计
1普通计数 2唯一计数 3累加计数 4时间差 5同周期计数
1 s_user_count_in_sample = groupby user count instance 
2 s_user_dist_count_item = groupby user distinct count item(shop,item_brand,cp_2_head等factor)
3 s_user_cum_count_for = groupby user cumsum count forward
3 s_user_cum_count_bac = groupby user cumsum count backward
4 s_user_time_interval_for = groupby user diff timestamp forward
4 s_user_time_interval_bac = groupby user diff timestamp backward
5 s_user_count_in_1_hour_for = groupby user cumsum count in hour forward
5 s_user_count_in_1_hour_bac = groupby user cumsum count in hour backward
#1,2
t1_group = sample_data.groupby("user_id")
t1 = t1_group[["instance_id"]].count()
t1.rename(columns={"instance_id":"s_user_count_in_sample"},inplace=True)
t1["s_user_dist_count_item"] = t1_group["item_id"].agg(lambda x:x.unique().size)

#4
#仅获取有重复的userid
tmp_dulicated = tmp_data.duplicated(subset="user_id", keep=False)
tmp_dulicated_data = tmp_data.loc[tmp_dulicated, :]
#剔除重复user中时间戳重复的样本，后续merge后时间戳相同的样本时间间隔也相同
tmp_drop_usertime = tmp_dulicated_data.drop_duplicates(subset=["user_id", "context_timestamp"])
tmp_drop_usertime.sort_values(["user_id", "context_timestamp"], ascending=[1,1], inplace=True)
tmp_drop_usertime["user_last_click_time_interval"] = tmp_drop_usertime.groupby("user_id")["context_timestamp"].diff(periods=1)
tmp_drop_usertime["user_next_click_time_interval"] = -tmp_drop_usertime.groupby("user_id")["context_timestamp"].diff(periods=-1)
tmp_dulicated_data = pd.merge(tmp_dulicated_data, tmp_drop_usertime[["user_last_click_time_interval", "user_next_click_time_interval","user_id", "context_timestamp"]], how="left", on=["user_id", "context_timestamp"])
action_v2_feature = pd.merge(sample_id[["instance_id"]], tmp_dulicated_data[["instance_id", "user_last_click_time_interval", "user_next_click_time_interval"]], how="left", on="instance_id")
